In [11]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

model_id = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
).to(0)

processor = AutoProcessor.from_pretrained(model_id)

# Define a chat history and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image") 
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "What are these?"},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

# Ouvre une image avec PIL
image = Image.open("images/cubes.png").convert("RGB")

display(image)  # Affiche l'image pour vérification

inputs = processor(images=image, text=prompt, return_tensors='pt').to(0, torch.float16)

output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
print(processor.decode(output[0][2:], skip_special_tokens=True))


KeyboardInterrupt: 

In [ ]:


# Prépare l'input : image + question
inputs = processor(images=image, text="Qu'est-ce qu'il y a sur cette image ?", return_tensors="pt").to(model.device)

# Génère la réponse
outputs = model.generate(**inputs, max_new_tokens=128)

# Décodage du texte généré
answer = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Réponse :", answer)
